<a href="https://colab.research.google.com/github/theBuluku/Mchezo_Pesa_Limited_Super_Project./blob/master/Mchezo_Pesa_Limited_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing libraries.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import chi2,bartlett
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,KFold,StratifiedKFold,cross_val_score
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score,f1_score,classificatio

# PROBLEM STATEMENT.


A Prediction result of a game between team 1 and team 2, based on who's home and who's away, and on whether or not the game is friendly (include rank in your training).

#Hypothesis

#Metrics of success.

In [0]:
# Importing libraries.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import chi2,bartlett
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,KFold,StratifiedKFold,cross_val_score
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score,f1_score,classification_report,confusion_matrix
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

#Exploratory Data Analysis.

In [0]:
rankings=pd.read_csv('fifa_ranking.csv')

In [0]:
results=pd.read_csv('results.csv')

In [0]:
rankings.head(6)

In [0]:
results.head(6)

# Data statistics

In [0]:
#Provides basic statistics for the attributes-e,g counts percentiles,
#mean,standard deviation
results.describe()

In [0]:
rankings.describe()

In [0]:
#I found out the total number of rows and columns in the data set using “.shape”.
rankings.shape

In [0]:
results.shape

In [0]:
#It is also a good practice to know the columns and their corresponding data types,along with finding whether they contain null values or not.
rankings.info()

In [0]:
results.info()

In [0]:
rankings.columns

In [0]:
results.columns

In [0]:
# Checking for null values in the rank dataset

rankings.isnull().sum().any()

In [0]:
# Checking for null values in the rank dataset

results.isnull().sum().any()

In [0]:
# Checking for duplicates
rankings.duplicated().any()

In [0]:
# Checking for duplicates
results.duplicated().any()

In [0]:
# Droping duplicates
rankings.drop_duplicates(keep ='first',inplace=True)

# Checking for duplicates whether they have actually been dropped
rankings.duplicated().any()

In [0]:
# Getting the number of unique countries in the rank datasets

uniq_country = rankings.country_full.unique()
len(uniq_country)

In [0]:
# Getting the number of unique dates in the rank datasets

uniq_dates = rankings.rank_date.unique()
len(uniq_dates)

In [0]:
# Getting the number of unique dates in the rank datasets

uniq_dates = results.date.unique()
len(uniq_dates)

In [0]:
# Getting the number of unique dates in the rank datasets

uniq_dates = rankings.rank_date.unique()
len(uniq_dates)

#Combination of the Two Datasets.

In [0]:
# Checking unique dates that are common in both datasets

list1 = []

uniq_dates  = list(uniq_dates)
uniq_date_results = list(uniq_date_results)

for x in uniq_dates:
  if x in uniq_date_results:
    list1.append(x)
print(list1)
print(len(list1))

In [0]:
# Renaming the two columns for easier merging with the results datsets

rankings.rename(columns = {"country_full":"country","rank_date":"date"},inplace=True)
rankings.columns

In [0]:
# Merging the datasets using both the dates and country columns

fifa_df=pd.merge(rankings,results,on = ["date","country"])

# Previewing the merged dataset
fifa_df.head()
 

In [0]:
# Getting the statistical information of the merged dataset
fifa_df.describe()

In [0]:
# Getting the names of the columns in the merged dataset

fifa_df.columns

# Checking for Outliers.

In [0]:
# checking for outlers
fifa_df['away_score'].plot(kind = 'box')

In [0]:
# checking for outlers of homescore

fifa_df['home_score'].plot(kind = 'box')

#VISUALISATION.